In [1]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [2]:
#features
n2v_start = np.load("start_emb_128.npy")
n2v_end = np.load("end_emb_128.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("nodetype.npy")
#target
meanspeed = np.load("meanspeed_only.npy")

In [5]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=64, random_state=0,perplexity=50)


In [6]:
print(n2v_start)
print(n2v_end)

[[ 0.72621435  0.0965858   0.0142382  ... -0.43758184 -2.090535
   0.5017583 ]
 [ 0.7366812   0.13260578  0.07979007 ... -0.47865644 -2.1874576
   0.5778666 ]
 [ 0.7538181   0.2126736   0.01767093 ... -0.37269926 -1.8463417
   0.55626845]
 ...
 [-1.1139764  -0.7462887  -0.39946842 ... -0.27099925 -0.3327773
  -2.7682035 ]
 [ 0.7054061  -1.4814119  -1.5285032  ...  1.1054153  -0.46722254
  -1.2053367 ]
 [ 0.25418708 -0.36718926 -1.5011023  ...  1.0120728  -0.7492262
  -1.0215808 ]]
[[ 0.37035462  0.01037511  0.01786758 ... -0.2290353  -1.1111445
   0.2672848 ]
 [ 0.72621435  0.0965858   0.0142382  ... -0.43758184 -2.090535
   0.5017583 ]
 [ 0.7366812   0.13260578  0.07979007 ... -0.47865644 -2.1874576
   0.5778666 ]
 ...
 [-1.9747239  -0.91884375 -0.775578   ... -0.5645902  -0.42994672
  -4.050164  ]
 [ 0.25418708 -0.36718926 -1.5011023  ...  1.0120728  -0.7492262
  -1.0215808 ]
 [ 0.7054061  -1.4814119  -1.5285032  ...  1.1054153  -0.46722254
  -1.2053367 ]]


In [7]:
n2v = np.concatenate((n2v_start, n2v_end), axis=1)
print(n2v)

[[ 0.72621435  0.0965858   0.0142382  ... -0.2290353  -1.1111445
   0.2672848 ]
 [ 0.7366812   0.13260578  0.07979007 ... -0.43758184 -2.090535
   0.5017583 ]
 [ 0.7538181   0.2126736   0.01767093 ... -0.47865644 -2.1874576
   0.5778666 ]
 ...
 [-1.1139764  -0.7462887  -0.39946842 ... -0.5645902  -0.42994672
  -4.050164  ]
 [ 0.7054061  -1.4814119  -1.5285032  ...  1.0120728  -0.7492262
  -1.0215808 ]
 [ 0.25418708 -0.36718926 -1.5011023  ...  1.1054153  -0.46722254
  -1.2053367 ]]


In [9]:
from sklearn.decomposition import PCA
pca = PCA(n_components=64)

PCA(n_components=64)

In [11]:
pca.fit(n2v)
n2v_64 = pca.transform(n2v)
n2v_64

array([[-1.3989583 , -2.5385208 ,  1.77042   , ..., -1.0551376 ,
        -0.54207414,  0.6368675 ],
       [-1.8891281 , -3.4466548 ,  2.3741302 , ..., -1.364933  ,
        -0.74396974,  0.9243126 ],
       [-1.8135501 , -3.2302313 ,  2.2440476 , ..., -1.2673515 ,
        -0.83842057,  1.0760049 ],
       ...,
       [-0.7294558 ,  0.29150707, -1.751305  , ..., -1.6388096 ,
         1.0252062 , -0.5843897 ],
       [ 0.8310536 , -1.7252822 , -0.93003094, ...,  0.5347019 ,
         0.52981085, -1.0050435 ],
       [ 0.83132625, -1.7260897 , -0.9254368 , ...,  0.5331711 ,
         0.5326352 , -1.005796  ]], dtype=float32)

In [38]:
np.size(Y_train,0)

86992

In [14]:
infras = np.concatenate((lanes,oneway,roadtype,maxspeed,nodetype),axis = 1)

In [22]:
np.set_printoptions(threshold=1000)
np.set_printoptions(edgeitems=3)
infras

array([[3, 0, 1, ..., 0, 0, 0],
       [3, 0, 1, ..., 0, 0, 0],
       [3, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [4, 1, 0, ..., 0, 0, 0],
       [4, 1, 0, ..., 0, 0, 0]])

In [28]:
feature = np.concatenate((n2v_64, infras), axis=1)
feature

array([[-1.39895833, -2.53852081,  1.77041996, ...,  0.        ,
         0.        ,  0.        ],
       [-1.88912809, -3.4466548 ,  2.37413025, ...,  0.        ,
         0.        ,  0.        ],
       [-1.81355011, -3.23023129,  2.24404764, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.72945583,  0.29150707, -1.75130498, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.83105361, -1.72528219, -0.93003094, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.83132625, -1.72608972, -0.92543679, ...,  0.        ,
         0.        ,  0.        ]])

In [31]:
pca2 = PCA(n_components=32)
pca2.fit(feature)
feature_32 = pca2.transform(feature)

In [32]:
feature_32

array([[ 4.94915564e+01, -1.36051361e+00, -2.49776842e+00, ...,
         8.52741896e-01,  1.13475836e-02, -1.26863707e-01],
       [ 4.95063552e+01, -1.84366039e+00, -3.42026072e+00, ...,
         1.08871305e+00,  4.06659754e-02, -1.51682915e-01],
       [ 4.95062709e+01, -1.76955659e+00, -3.19388251e+00, ...,
         1.10960686e+00,  9.41782769e-02, -2.71976084e-01],
       ...,
       [-5.36852183e-01, -7.53016145e-01,  2.38892099e-01, ...,
        -1.13386927e+00, -1.15249425e+00, -2.13044007e-01],
       [-2.04758298e+01,  8.28318071e-01, -1.66039748e+00, ...,
        -2.53217681e+00, -1.16215905e+00,  1.53252629e-01],
       [-2.04758422e+01,  8.27649429e-01, -1.65587675e+00, ...,
        -2.53429460e+00, -1.15116110e+00,  1.43757850e-01]])

In [2]:
from sklearn.model_selection import train_test_split

#split
n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, meanspeed_train, meanspeed_test = train_test_split(n2v_start, n2v_end, lanes,oneway,roadtype,maxspeed,nodetype,meanspeed)
del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del meanspeed

In [3]:
embinplist = list()
inplist = list()

n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
embinplist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
embinplist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(feature_32,meanspeed)

In [39]:
embinplist = list()
inplist = list()

feature_inp = Input((32,))
feature_reg = Dense(32,activation = "relu", use_bias = True)(feature_inp)
inplist.append(feature_inp)
embinplist.append(feature_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ValueError: A `Concatenate` layer should be called on a list of at least 2 inputs

In [4]:
emb.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 2)]          0                                            
______________________________________________________________________________________________

In [5]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999)
emb.compile(optimizer=adam, loss='mae',metrics=['mae'])

In [6]:
emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train],
        meanspeed_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test],
        meanspeed_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 4s 43us/sample - loss: 9.1310 - mean_absolute_error: 9.1310 - val_loss: 7.3224 - val_mean_absolute_error: 7.3224
Epoch 2/100
86992/86992 [==============================] - 3s 34us/sample - loss: 6.6397 - mean_absolute_error: 6.6397 - val_loss: 6.3645 - val_mean_absolute_error: 6.3645
Epoch 3/100
86992/86992 [==============================] - 3s 34us/sample - loss: 5.8340 - mean_absolute_error: 5.8340 - val_loss: 5.7707 - val_mean_absolute_error: 5.7707
Epoch 4/100
86992/86992 [==============================] - 3s 34us/sample - loss: 5.2389 - mean_absolute_error: 5.2389 - val_loss: 5.6245 - val_mean_absolute_error: 5.6245
Epoch 5/100
86992/86992 [==============================] - 3s 34us/sample - loss: 4.8846 - mean_absolute_error: 4.8847 - val_loss: 5.2896 - val_mean_absolute_error: 5.2896
Epoch 6/100
86992/86992 [==============================] - 3s 35us/sample - loss: 4.6349 -

86992/86992 [==============================] - 3s 35us/sample - loss: 2.4902 - mean_absolute_error: 2.4902 - val_loss: 4.0690 - val_mean_absolute_error: 4.0690
Epoch 96/100
86992/86992 [==============================] - 3s 34us/sample - loss: 2.5027 - mean_absolute_error: 2.5027 - val_loss: 4.0883 - val_mean_absolute_error: 4.0883
Epoch 97/100
86992/86992 [==============================] - 3s 35us/sample - loss: 2.4986 - mean_absolute_error: 2.4986 - val_loss: 4.0712 - val_mean_absolute_error: 4.0712
Epoch 98/100
86992/86992 [==============================] - 3s 34us/sample - loss: 2.4915 - mean_absolute_error: 2.4915 - val_loss: 4.0748 - val_mean_absolute_error: 4.0748
Epoch 99/100
86992/86992 [==============================] - 3s 32us/sample - loss: 2.4767 - mean_absolute_error: 2.4767 - val_loss: 4.0534 - val_mean_absolute_error: 4.0534
Epoch 100/100
86992/86992 [==============================] - 3s 32us/sample - loss: 2.4760 - mean_absolute_error: 2.4760 - val_loss: 4.0856 - val_me

In [49]:
emb.save('/home/fyp2/Ideaer/Madrid/model/all')